In [1]:
import os
import numpy as np
import pandas as pd
import mol2graph as mg
import shutil
from rdkit import Chem
from mol2vec.features import mol2alt_sentence, MolSentence
from gensim.models import word2vec
import json
import pickle
import math
import collections
from collections import OrderedDict
import sys

In [2]:
def pdb2seq(pdb_file):
    letters = {'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C', 'GLU': 'E', 'GLN': 'Q', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P', 'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'}
    prev = '-1'
    res = []
    with open(pdb_file) as input_file:
        for line in input_file:
            if len(line) < 1: continue
            if line[0:6].strip() != 'ATOM': continue
            if line[21:27].strip() != prev:
                res.append(letters[line[17:20]])
            prev = line[21:27].strip()
    return ''.join(res)

def pdb2mol(in_path,file,out_path):
    pocket = "obabel -ipdb {} -omol2 -O {}".format(os.path.join(in_path,file+'_pocket.pdb'),os.path.join(out_path,file+'_pocket.mol2'))
    os.system(pocket)
    
def sentence2vec(sentence, model, unseen=None):
    keys = set(model.wv.key_to_index.keys())
    if unseen:
        unseen_vec = model.wv.word_vec(unseen)
        res = sum([model.wv.word_vec(y) if y in set(sentence) & keys else unseen_vec for y in sentence])
    else:
        res = sum([model.wv.word_vec(y) for y in sentence if y in set(sentence) & keys])
    return res

def smi2vec(smi, model):
    mol = Chem.MolFromSmiles(smi)
    sentence = MolSentence(mol2alt_sentence(mol, 1))
    mol_vec = sentence2vec(sentence, model, unseen='UNK')
    return mol_vec

def get_amino_vecs(model):
    amino_smi = {
        'A': 'CC(N)C(=O)O',
        'R': 'N=C(N)NCCCC(N)C(=O)O',
        'N': 'NC(=O)CC(N)C(=O)O',
        'D': 'NC(CC(=O)O)C(=O)O',
        'C': 'NC(CS)C(=O)O',
        'E': 'NC(CCC(=O)O)C(=O)O',
        'Q': 'NC(=O)CCC(N)C(=O)O',
        'G': 'NCC(=O)O',
        'H': 'NC(Cc1cnc[nH]1)C(=O)O',
        'I': 'CCC(C)C(N)C(=O)O',
        'L': 'CC(C)CC(N)C(=O)O',
        'K': 'NCCCCC(N)C(=O)O',
        'M': 'CSCCC(N)C(=O)O',
        'F': 'NC(Cc1ccccc1)C(=O)O',
        'P': 'O=C(O)C1CCCN1',
        'S': 'NC(CO)C(=O)O',
        'T': 'CC(O)C(N)C(=O)O',
        'W': 'NC(Cc1c[nH]c2ccccc12)C(=O)O',
        'Y': 'NC(Cc1ccc(O)cc1)C(=O)O',
        'V': 'CC(C)C(N)C(=O)O'
    }
    amino_vec = {}
    for amino in amino_smi:
        amino_vec[amino] = smi2vec(amino_smi[amino], model)
    return pd.DataFrame(amino_vec).T

def seq2vec(seq, amino_vecs):
    vecs = np.array(amino_vecs.loc[list(seq)])
    return np.mean(vecs, axis=0)


In [3]:

def extract_data(fpath,x):
    ligands = json.load(open(fpath+"ligands_can.txt"), object_pairs_hook=OrderedDict)
    proteins = json.load(open(fpath+"proteins.txt"), object_pairs_hook=OrderedDict)
    Y = pickle.load(open(fpath + "Y","rb"), encoding='latin1') ### TODO: read from raw
    print(os.path.basename(fpath).lower())
    
    if x == 'davis':
        Y = -(np.log10(Y/(math.pow(10,9))))
        uni_data = pd.read_csv(fpath+'uniprot_gi_map_cleaned.csv')
        uni_id = [str(row['uniprot']) for idx,row in uni_data.iterrows()]
        pocket_failed = [str(row['pocket structure']) for idx,row in uni_data.iterrows()]
    print(pocket_failed[0:10])
    compounds,protein_seq,y = [],[],[]
    pro_id,com_id = [],[]
    for d_idx,(drug,smile) in enumerate(ligands.items()):
        for p_idx,(protein,fasta) in enumerate(proteins.items()):
            if math.isnan(Y[d_idx][p_idx]):
                continue
            if x == 'davis':
                if uni_id[p_idx] == 'nan' or pocket_failed[p_idx] == 'FALSE':
                    print(' null ')
                    continue
                pro_id.append(uni_id[p_idx])
            else:
                pro_id.append(protein)
            
            compounds.append(smile)
            protein_seq.append(fasta)
            y.append(Y[d_idx][p_idx])
            com_id.append(drug)
            
            
    if x == 'kiba':
        y = [-i for i in y]
        min_val = min(y)
        y= [ i-min_val for i in y]
        
    return compounds,protein_seq,y,pro_id,com_id


In [4]:

def extract_pocket(dataset,deeppocket_path): 
    # err_id = set()  
    # data_path = os.path.join(os.path.dirname(code_dir),dataset,'protein')
    cnt = 0
    code_dir = os.getcwd()
    data_path = "{}/{}/{}".format(os.path.dirname(code_dir),'datasets',dataset)
    pdb_list = os.listdir(os.path.join(data_path,'protein'))

    if not os.path.exists(os.path.join(data_path,'pocket')):
        os.mkdir(os.path.join(data_path,'pocket'))

    os.chdir(deeppocket_path)
    print(pdb_list)
    for pdb in pdb_list:
        uniprot_id = pdb.split('.')[0]
        predict_command = "python predict.py -p {} -c first_model_fold1_best_test_auc_85001.pth.tar -s seg0_best_test_IOU_91.pth.tar".format(os.path.join(data_path,'protein',pdb))
        i = os.system(predict_command)
        print(i)
        
        if not os.path.exists(os.path.join(data_path,'protein',uniprot_id+'_nowat_pocket1.pdb')):
            # err_id.add(uniprot_id)
            print('pocket prediction faild in '+uniprot_id)
            continue

        os.rename(os.path.join(data_path,'protein',uniprot_id+'_nowat_pocket1.pdb'),os.path.join(data_path,'protein',uniprot_id+'_pocket.pdb'))
        shutil.move(os.path.join(data_path,'protein',uniprot_id+'_pocket.pdb'),os.path.join(data_path,'pocket',uniprot_id+'_pocket.pdb'))
        for i in ['_nowat_1.dx','_nowat.gninatypes','_nowat.pdb']:
            file = "{}/{}{}".format(os.path.join(data_path,'protein'),uniprot_id,i)
            os.remove(file)
        nowat_dir="{}/{}{}".format(os.path.join(data_path,'protein'),uniprot_id,'_nowat_out')
        shutil.rmtree(nowat_dir)
    
    os.chdir(code_dir)
    # return err_id

# prepare dataset

In [3]:
path = '../datasets'
datasets = ['PDBbind','kiba','davis']
#select dataset default value: PDBbind
x=datasets[0]


In [4]:
if x == 'PDBbind':
    #extract pocket sequence 
    protein_seq = {}
    pocket_seq = {}
    for pdb_path in ['../datasets/PDBbind/refined-set', '../datasets/PDBbind/general-set']:
        for name in os.listdir(pdb_path):
            if name in ['index', 'readme']:
                continue
            protein_pdbfile = os.path.join(pdb_path, name, name + '_protein.pdb')
            pocket_pdbfile = os.path.join(pdb_path, name, name + '_pocket.pdb')

            protein_seq[name] = pdb2seq(protein_pdbfile)
            pocket_seq[name] = pdb2seq(pocket_pdbfile)

    data = pd.read_excel('../datasets/PDBbind/general_data.xlsx', skiprows=1)
    data = data[~(data['Resolution']=="NMR")&~(data['Ligand Name'].str.contains("-mer"))]
    data['protein_seq'] = data['PDB code'].apply(lambda x : protein_seq[x])
    data['pocket_seq'] = data['PDB code'].apply(lambda x : pocket_seq[x])
    data = data[['PDB code', 'Canonical SMILES', 'protein_seq', 'pocket_seq', 'pKd pKi pIC50']]
    data.rename(columns={'PDB code':'protein_id', 'Canonical SMILES':'compound', 'pKd pKi pIC50':'label'},inplace=True)
    cols = list(data)
    cols.insert(3,cols.pop(cols.index('compound')))

else:
    data = pd.DataFrame()
    pocket_seq = {}
    fpath = "../datasets/{}/".format(x)
    
    compounds,protein_seq,label,pdb_id,compound_id = extract_data(fpath,x)
    #extract pocket
    pocket_failed_set = extract_pocket(x,'./DeepPocket-main/')
    print(pocket_failed_set)
    
    for name in os.listdir(os.path.join(fpath,'pocket')):
        pocket_pdbfile = os.path.join(fpath, 'pocket', name)
        pocket_seq[name] = pdb2seq(pocket_pdbfile)
    
    data['protein_id'] = pdb_id
    data['compound_id'] = compound_id
    data['protein_seq'] = protein_seq
    data['pocket_seq'] = data['protein_id'].apply(lambda x : pocket_seq[x])
    data['compound'] = compounds
    data['label'] = label


for idx, row in data.iterrows():
    code = row['protein_id']
    if len(row['pocket_seq']) <= 30:
        # short_set.add(code)
        data.loc[idx,"drop"] = 'NA'
        continue    
# print(short_set)

data = data.dropna()
data.to_csv("{}/{}_input_data.csv".format(path,x), index=None, sep=',')    


# prepare smi2vec vec


In [ ]:
model = word2vec.Word2Vec.load('mol2vec_300dim.pkl')
amino_vecs_pd = get_amino_vecs(model)

com_vec_pd = pd.DataFrame()
com_vec_pd['vec'] = data['compound'].apply(lambda smi: smi2vec(smi, model))
compound_vector = np.array([vec for vec in com_vec_pd['vec']])

poc_vec_pd = pd.DataFrame()
poc_vec_pd['vec'] = data['pocket_seq'].apply(lambda seq: seq2vec(seq, amino_vecs_pd))
pocket_vector = np.array([vec for vec in poc_vec_pd['vec']])

# prepare graph2vec

In [9]:
dataset_path = "../datasets/{}".format(x)
mol2_path = ["{}/{}/{}-set".format(dataset_path,'mol2','general'),"{}/{}".format(dataset_path,'pocket')]
graph_path = "{}/{}/{}".format(dataset_path,'graph','pocket')
graph_csv = "{}/{}".format(dataset_path,'pocket_graph.csv')

In [ ]:
for idx, row in data.iterrows():
    if x == 'PDBbind':
        code = row['protein_id']
        set = 'general'
        if not os.path.exists(os.path.join(dataset_path,'general-set',code)):
            set = 'refined' 
        in_path = "{}/{}-set/{}".format(dataset_path,set,code)
        out_path = "{}/{}/{}-set".format(dataset_path,'mol2',set)
        
    else:
        in_path = "{}/{}".format(dataset_path,'pocket') 
        out_path = "{}/{}".format(dataset_path,'mol2')
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    pdb2mol(in_path,code,out_path)

In [ ]:
mg.main(mol2_path[0],graph_path,"aa",False)
g2v_command = "python graph2vec.py --input-path {} --output-path {}".format(graph_path,graph_csv)
os.system(g2v_command)

In [14]:
pdb_id = list(data['protein_id'])
graph_data = pd.read_csv(graph_csv)
graph_dict = {}

if x == 'PDBbind':
    graph_dict = {str(row['type']).split('_')[0]:row[1:] for idx,row in graph_data.iterrows()}
    
    graph_data_new = pd.DataFrame.from_dict(graph_dict,orient='index')
    graph_data_new.reset_index(inplace=True)
    graph_data_new.rename(columns={"index":"type"},inplace=True)
else:
    pdb_name = str(row['type']).split('_')[0]
    graph_dict[pdb_name] = row[1:].to_list()    

graph_vec = np.array([graph_dict[i].tolist() for i in pdb_id if i in graph_dict.keys()])

# prepare protr

In [ ]:
protr ="Rscript protor.r {} {}".format('~/StackCPA/datasets/PDBbind_input_data.csv','~/StackCPA/datasets/PDBbind_input_protr.csv')
os.system(protr)

In [20]:
protr_vec = pd.read_csv('../datasets/{}_input_protr.csv'.format(x),skiprows=0,sep=',')
protr_vec = protr_vec.iloc[:,1:]
protr_vec = [row.tolist() for idx,row in protr_vec.iterrows()]
protr_vector = np.array(protr_vec)
protr_vector.shape

In [ ]:
X_poc = np.hstack((compound_vector, pocket_vector,graph_vec,protr_vector))
y = np.array([x for x in data['label']])

np.save('../data/X.npy', X_poc)
np.save('../data/y.npy', y)

X_poc.shape, y.shape